In [196]:
import time
import numba
import numpy as np
from numba import guvectorize
from numba import guvectorize, float32, float64

In [197]:
# The target keyword decorator for loss and error return

def gu_loss(func, *args, **kwargs):
    """
    The guvectorize decorated method, runs in nopython mode
    :param func: a loss error method
    :param target: choose between | 1: None -> serial | 2: 'parallel' -> parallel | execution method
    :return: the decorated loss, error function
    """
    kwargs_ = {k: v for k, v in kwargs.items() if v is not None}
    return guvectorize([(float32[:], float32[:], float32[:], float32[:]),
                      (float64[:], float64[:], float64[:], float64[:])],
                     '(n),(n)->(),(n)', nopython=True, *args, **kwargs_)(func)

In [198]:
def mse(x, y, loss, error):
    for i in range(y.shape[0]):
        error[i] = y[i] - x[i]
    loss[0] = 0.5 * np.sum(error[:]**2)

def cross_entropy(x, y, loss, derivation):
    l = np.log(y[:])
    for i in range(y.shape[0]):
        derivation[i] = -(x[i]/y[i])
    loss[0] = -np.sum(x[:] * l)

def mse_py(yd, yp):
    e = yp - yd
    loss = 0.5 * np.sum(e**2)
    return loss, e

def xentropy(yd, yp):
    l = np.log(yp)
    loss = -np.sum(yd * l)
    deriv = -(yd/yp)
    return loss, deriv

In [199]:
a = np.arange(5, dtype=np.float32)
b = np.arange(10, 15, dtype=np.float32)

a, b

(array([0., 1., 2., 3., 4.]), array([10., 11., 12., 13., 14.]))

In [200]:
s_mse = gu_loss(mse)
p_mse = gu_loss(mse, target="parallel")
s_ce = gu_loss(cross_entropy)
p_ce = gu_loss(cross_entropy, target="parallel")


In [201]:
def ntime(func1):
    start = time.time()
    func1(a, b)
    end = time.time()
    print("Elapsed Numba Pre = %s" % (end - start))
    start = time.time()
    f2 = func1(a, b)
    end = time.time()
    print("Elapsed Numba Post = %s" % (end - start))
    return f2

def ptime(func1):
    start = time.time()
    f1 = func1(a, b)
    end = time.time()
    print("Elapsed PyTime = %s" % (end - start))
    return f1

In [202]:
print("MSE")
mse1 = ntime(s_mse)
mse2 = ptime(mse_py)
mse1, mse2



MSE
Elapsed Numba Pre = 0.0
Elapsed Numba Post = 0.0
Elapsed PyTime = 0.0


((250.0, array([10., 10., 10., 10., 10.])),
 (250.0, array([10., 10., 10., 10., 10.])))

In [203]:
print("CE")
ce1 = ntime(s_ce)
ce2 = ptime(xentropy)
ce1, ce2

CE
Elapsed Numba Pre = 0.0
Elapsed Numba Post = 0.0
Elapsed PyTime = 0.0


((-25.618785963220013,
  array([-0.        , -0.09090909, -0.16666667, -0.23076923, -0.28571429])),
 (-25.618785963220013,
  array([-0.        , -0.09090909, -0.16666667, -0.23076923, -0.28571429])))